In [0]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 126.2MB 97kB/s 
  Created wheel for h2o: filename=h2o-3.28.1.2-py2.py3-none-any.whl size=126306247 sha256=9481246190a7d8b8f9effc5fd9b906d3fff3e19b9076c9d7d3c4dfa4adb865d1
  Stored in directory: /root/.cache/pip/wheels/18/3c/ca/07a89a0183a77f66ad2fac339e16ba5f943dde887e9a1cac18
Successfully built h2o


In [0]:
import pandas as pd
import numpy as np
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
from google.colab import files
uploaded = files.upload()

import io
data = pd.read_csv(io.BytesIO(uploaded['cleaned_data_reduced_feats.csv']))

Saving cleaned_data_reduced_feats.csv to cleaned_data_reduced_feats.csv


In [0]:
cont_columns = ['Start_Lat', 'Start_Lng', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']

for c in data.columns:
    if c not in cont_columns:
        data[c] = data[c].astype('category')

In [0]:
#Splitting into training and validation sets
feats = data.drop('Severity', axis = 1)
target = data['Severity']

x_train, x_validate, y_train, y_validate = train_test_split(feats, target, random_state = 11, test_size = .4)

In [0]:
#Seeing results for synthetically balancing minority class samples
sm = SMOTE(random_state = 11, n_jobs = -1, sampling_strategy = {1 : 250000, 2 : 420000, 3: 400000, 4: 400000})

x_train_res, y_train_res = sm.fit_resample(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (420000) in class 2 will be larger than the number of samples in the majority class (class #2 -> 405785)
  n_samples_majority))
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=Futu

In [0]:
h2o.init(nthreads = -1)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8rxta7e5
  JVM stdout: /tmp/tmp8rxta7e5/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp8rxta7e5/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.1.2
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_unknownUser_fgd0g5
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
y_train_res = pd.DataFrame(y_train_res)
y_validate = pd.DataFrame(y_validate)

In [0]:
h2o_res_train = h2o.H2OFrame(x_train_res)
h2o_res_target = h2o.H2OFrame(y_train_res)
h2o_valid = h2o.H2OFrame(x_validate)
h2o_valid_target = h2o.H2OFrame(y_validate)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
#Ensuring columns have the same names and categorical columns are encoded as such
h2o_res_train.columns = list(feats.columns)
h2o_res_target.columns = ['Severity']

h2o_res_target['Severity'] = h2o_res_target['Severity'].asfactor()
h2o_valid_target['Severity'] = h2o_valid_target['Severity'].asfactor()

#All these have the same column names
for c in h2o_res_train.columns:
  if c not in cont_columns:
    h2o_res_train[c] = h2o_res_train[c].ascharacter().asfactor()
    h2o_valid[c] = h2o_valid[c].asfactor()

        

In [0]:
train_final = h2o_res_train
valid_final = h2o_valid

#Need to put the target column with the data for how h2o operates
train_final['Severity'] = h2o_res_target
valid_final['Severity'] = h2o_valid_target

In [0]:
rf_res_1 = H2ORandomForestEstimator(ntrees = 50, max_depth = 10, categorical_encoding = 'enum', nfolds = 4)

features = list(train_final.columns)
features.remove('Severity')

rf_res_1.train(x = features, y = 'Severity', training_frame = train_final, validation_frame = valid_final)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [0]:
rf_res_1

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1585279050703_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,200.0,1242349.0,10.0,10.0,10.0,241.0,741.0,478.005




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.14950191817155745
RMSE: 0.38665477906209494
LogLoss: 0.47942142474241317
Mean Per-Class Error: 0.1475882738095238

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,1,2,3,4,Error,Rate
0,220600.0,27142.0,772.0,1486.0,0.117600,"29,400 / 250,000"
1,9041.0,388274.0,16888.0,5797.0,0.075538,"31,726 / 420,000"
2,1197.0,17442.0,281900.0,99461.0,0.295250,"118,100 / 400,000"
3,548.0,822.0,39416.0,359214.0,0.101965,"40,786 / 400,000"
4,231386.0,433680.0,338976.0,465958.0,0.149668,"220,012 / 1,470,000"



Top-4 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-03-27 04:04:52,42 min 1.542 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-03-27 04:05:07,42 min 16.638 sec,2.0,0.394148,0.504136,0.193902,0.366998,0.457097,0.146135
2,,2020-03-27 04:05:28,42 min 37.475 sec,5.0,0.389470,0.490918,0.180767,0.364402,0.455822,0.136865
3,,2020-03-27 04:06:02,43 min 12.326 sec,10.0,0.390516,0.486973,0.166314,0.370137,0.458930,0.133695
4,,2020-03-27 04:06:59,44 min 9.056 sec,18.0,0.389563,0.483944,0.156518,0.373523,0.463128,0.131017
5,,2020-03-27 04:08:31,45 min 40.883 sec,31.0,0.388103,0.481487,0.152802,0.373940,0.464575,0.129720
6,,2020-03-27 04:10:46,47 min 56.261 sec,50.0,0.386655,0.479421,0.149668,0.372730,0.463133,0.128759



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Interstate_Hwy,6.663538e+06,1.000000,0.319397
1,road_blocked,2.640418e+06,0.396249,0.126561
2,Side,1.940091e+06,0.291150,0.092993
3,State_Hwy,1.920443e+06,0.288202,0.092051
4,Traffic_Signal,1.450231e+06,0.217637,0.069513
5,Pressure(in),8.425494e+05,0.126442,0.040385
6,Start_Lng,7.880222e+05,0.118259,0.037772
7,Timezone,6.671669e+05,0.100122,0.031979
8,Numbered_Hwy,6.366256e+05,0.095539,0.030515
9,Sunrise_Sunset,4.976254e+05,0.074679,0.023852



See the whole table with table.as_data_frame()


Trading off bias for variance produced significantly more accurate classifications of severe accidents at the expense of worse performance classifying level 3 accidents. Performance for the other classes was unchanged, and the model is still completely ignoring level 1 accidents. In practice, this is not as big a concern as it appears, as misclassifying an extremely minor accident as minor would not a very costly mistake to make. Results indicate that classes 3 and 4 are very similar, and emphasizes the importance of a next step to be discerning differences between these classes and engineering features that capture these. Text analysis of the description field is the likely scenario here.
